<a href="https://colab.research.google.com/github/quanggquangg/phone_number_predict/blob/main/LSTM_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [213]:
from google.colab import drive
import numpy as np
import pandas as pd

path = '/content/train_dataset_new.csv'
df = pd.read_csv(path)
df = df.drop(labels = 'Unnamed: 0', axis = 1)

In [214]:
df.head()

,price_vnd,sim_number,sim_price_range
0,450000,343189288,1
1,3000000,888899580,4
2,500000,928960006,2
3,5000000,902438679,5
4,450000,334307889,1


In [215]:
X = []
y = []

for i, r in df.iterrows():
  X.append([int(j) for j in str(r['sim_number'])])
  y.append(r["sim_price_range"])

X, y = np.array(X), np.array(y)

In [216]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y) 
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_one_hot = onehot_encoder.fit_transform(integer_encoded)
print(y_one_hot) 

[[0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [226]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

In [222]:
X_train.shape

(160000, 9)

In [225]:
y_train.shape

(160000,)

In [248]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import LSTM

regressor = Sequential()

regressor.add(
    LSTM(units = 64, 
              return_sequences = True, 
              input_shape = (X_train.shape[1],1))
             )
regressor.add(
    Dropout(0.2)
             )
regressor.add(
    LSTM(units = 32, 
              return_sequences = True)
             )
regressor.add(
    Dropout(0.2)
             )
regressor.add(
    LSTM(units = 16, 
              return_sequences = True)
             )
regressor.add(
    Dropout(0.2)
             )
regressor.add(
    LSTM(units = 8)
             )
regressor.add(
    Dropout(0.2)
             )
regressor.add (Dense(units=6, activation = "softmax"))

regressor.compile(
    optimizer = "adam", 
    loss = "categorical_crossentropy",
    metrics = ["accuracy"])

from tensorflow.keras.callbacks import ModelCheckpoint
model_checkpoint_callback = ModelCheckpoint(
    filepath="ckpt_best.hdf5",
    save_weights_only=True,
    monitor='val_accuracy',
    save_best_only=True, verbose=1)

history = regressor.fit(X_train, y_train, epochs = 10, batch_size = 64, validation_data = (X_test, y_test), callbacks = [model_checkpoint_callback])

Epoch 1/10
2498/2500 [============================>.] - ETA: 0s - loss: 1.4884 - accuracy: 0.4133
Epoch 1: val_accuracy improved from -inf to 0.52000, saving model to ckpt_best.hdf5
2500/2500 [==============================] - 59s 21ms/step - loss: 1.4882 - accuracy: 0.4134 - val_loss: 1.2687 - val_accuracy: 0.5200
Epoch 2/10
2498/2500 [============================>.] - ETA: 0s - loss: 1.2743 - accuracy: 0.5171
Epoch 2: val_accuracy improved from 0.52000 to 0.55658, saving model to ckpt_best.hdf5
2500/2500 [==============================] - 55s 22ms/step - loss: 1.2743 - accuracy: 0.5171 - val_loss: 1.1874 - val_accuracy: 0.5566
Epoch 3/10
2498/2500 [============================>.] - ETA: 0s - loss: 1.2108 - accuracy: 0.5432
Epoch 3: val_accuracy improved from 0.55658 to 0.57045, saving model to ckpt_best.hdf5
2500/2500 [==============================] - 52s 21ms/step - loss: 1.2109 - accuracy: 0.5432 - val_loss: 1.1259 - val_accuracy: 0.5705
Epoch 4/10
2498/2500 [=====================

In [256]:
test = pd.read_csv('/content/test.csv')

In [268]:
test.head()

,sim_number
0,776260543
1,822977488
2,707963979
3,344134986
4,795718118


In [269]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149060 entries, 0 to 149059
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   sim_number  149060 non-null  int64
dtypes: int64(1)
memory usage: 1.1 MB


In [266]:
T = []
for i, r in test.iterrows():
  T.append([int(j) for j in str(r['sim_number'])])

In [270]:
T = np.array(T)

In [295]:
T.shape

(149060, 9)

In [297]:
r = []
result = regressor.predict(T)
result.shape


4659/4659 [==============================] - 29s 6ms/step


(149060, 6)

In [300]:
index = []
for i in result:
  z = np.argmax(i)
  index.append(z)

In [301]:
print(index)

[3, 4, 0, 1, 3, 1, 0, 1, 0, 4, 1, 1, 3, 3, 1, 2, 2, 1, 1, 3, 0, 4, 0, 3, 1, 3, 3, 0, 2, 1, 0, 0, 1, 3, 1, 3, 1, 0, 0, 1, 1, 0, 3, 1, 2, 4, 4, 3, 1, 4, 0, 3, 0, 0, 0, 4, 4, 4, 3, 3, 0, 2, 1, 3, 1, 2, 0, 0, 0, 4, 2, 4, 0, 3, 3, 3, 1, 0, 3, 3, 3, 4, 1, 0, 0, 3, 2, 4, 0, 2, 2, 4, 1, 2, 4, 2, 0, 4, 2, 4, 1, 4, 0, 1, 1, 0, 1, 3, 0, 3, 2, 1, 3, 3, 1, 1, 1, 3, 1, 0, 3, 0, 1, 4, 3, 3, 3, 0, 0, 1, 1, 1, 0, 3, 3, 0, 4, 0, 1, 3, 2, 1, 1, 4, 3, 3, 1, 3, 1, 3, 0, 2, 1, 1, 1, 1, 4, 0, 2, 1, 1, 1, 1, 1, 0, 1, 3, 4, 3, 3, 5, 2, 3, 1, 1, 3, 0, 1, 0, 3, 4, 0, 1, 4, 1, 4, 3, 0, 3, 0, 3, 3, 2, 0, 1, 4, 1, 1, 0, 4, 3, 1, 3, 4, 1, 2, 1, 3, 2, 0, 1, 0, 2, 0, 1, 3, 1, 2, 2, 4, 1, 0, 1, 1, 1, 0, 3, 2, 0, 3, 2, 3, 4, 1, 2, 1, 3, 0, 0, 0, 3, 3, 0, 1, 1, 2, 2, 4, 1, 2, 2, 1, 1, 2, 0, 1, 0, 1, 1, 4, 0, 0, 2, 1, 1, 1, 3, 4, 3, 1, 1, 3, 2, 4, 0, 0, 1, 2, 1, 3, 3, 3, 0, 2, 3, 2, 2, 0, 2, 1, 2, 1, 1, 1, 3, 4, 4, 0, 3, 4, 3, 3, 3, 1, 3, 3, 1, 0, 0, 0, 4, 1, 2, 0, 0, 1, 1, 4, 1, 2, 3, 4, 4, 4, 0, 0, 1, 1, 0, 1, 4, 1, 0, 

In [302]:
test['sim_price_range'] = index

In [303]:
test.head()

,sim_number,sim_price_range
0,776260543,3
1,822977488,4
2,707963979,0
3,344134986,1
4,795718118,3


In [307]:
test.to_csv('/content/t.csv')

In [304]:
big_price = test[test['sim_price_range'] == 0 ]

In [305]:
big_price.head()

,sim_number,sim_price_range
2,707963979,0
6,818999191,0
8,585444777,0
20,378569788,0
22,386495666,0


In [306]:
big_price.to_csv('/content/big_price_test.csv')